# Логистическая регрессия

В этом задании мы предлагаем вам самостоятельно реализовать градиентный спуск.

В качестве метрики качества будем использовать AUC-ROC (Area Under ROC-Curve). Она предназначена для алгоритмов бинарной классификации, выдающих оценку принадлежности объекта к одному из классов. По сути, значение этой метрики является агрегацией показателей качества всех алгоритмов, которые можно получить, выбирая какой-либо порог для оценки принадлежности.

В Scikit-Learn метрика AUC реализована функцией sklearn.metrics.roc_auc_score. В качестве первого аргумента ей передается вектор истинных ответов, в качестве второго — вектор с оценками принадлежности объектов к первому классу.

### Инструкция по выполнению
    1. Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.
    2. Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!
    3. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).
    4. Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.
    5. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).
    6. Попробуйте поменять длину шага. Будет ли сходиться алгоритм, если делать более длинные шаги? Как меняется число итераций при уменьшении длины шага?
    7. Попробуйте менять начальное приближение. Влияет ли оно на что-нибудь?

In [1]:
import pandas
import sklearn 
from sklearn.metrics import roc_auc_score

1. Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [70]:
data = pandas.read_csv(r'data-logistic.csv',names=['Target','Sign1','Sign2'],index_col=False)
X=data.loc[:,'Sign1':]
y=data['Target']
df=pandas.read_csv(r'data-logistic.csv',header=None)

2. Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

In [44]:
def fw1(w1, w2, y, X, k, C):
    l = len(y)
    S = 0
    for i in xrange(0, l):
        S += y[i] * X['Sign1'][i] * (1.0 - 1.0 / (1.0 + math.exp(-y[i] * (w1*X['Sign1'][i] + w2*X['Sign2'][i]))))
    R=w1 + (k * (1.0 / l) * S) - k * C * w1 
    return R

In [45]:
def fw2(w1, w2, y, X, k, C):
    l = len(y)
    S = 0
    for i in xrange(0, l):
        S += y[i] * X['Sign2'][i] * (1.0 - 1.0 / (1.0 + math.exp(-y[i] * (w1*X['Sign1'][i] + w2*X['Sign2'][i]))))
    R = w2 + (k * (1.0 / l) * S) - k * C * w2 
    return R

In [43]:
C=0.0
w1=0.0
w2=0.0
k=0.1
#Test=fw2(w1, w2, y, X, k, C)
len(y)#205
X['Sign2'][200]

3.05393139864

3. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

In [66]:
import math
def grad(y, X, C=0.0, w1=0.0, w2=0.0, k=0.1, err=1e-5):
    i = 0
    i_max = 10000
    w1_new, w2_new = w1, w2

    while True:
        i += 1
        w1_new, w2_new = fw1(w1, w2, y, X, k, C), fw2(w1, w2, y, X, k, C)
        e = math.sqrt((w1_new - w1) ** 2 + (w2_new - w2) ** 2)

        if i >= i_max or e <= err:
            break
        else:
            w1, w2 = w1_new, w2_new

    return [w1_new, w2_new]

2 вариант функции

In [72]:
def GradientDescent( C, data):
    print('C=%d' % C)
    errorAccuracy=10**-5
    weights= [ 0.,0.]
    k=0.1#длина шага
    l= data[ 0].count()#количество элементов в выборке
    distance_euclidean=0

    weightsDelta=[0.,0.]
    for step in range(10000):#Рекомендуется ограничить сверху число итераций десятью тысячами.
        oldweightsDelta=weightsDelta
        weightsDelta=[0.,0.]

        for obj in data.values:
            y=obj[0]
            gradient=y*(1-1/(1+math.exp(-y*(weights[0]*obj[1]+weights[1]*obj[2]))))
            weightsDelta=list(map(lambda w,wd,x: wd+x*gradient-k*C*w,weights,weightsDelta,obj[1:]))

        #доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5)
        distance_euclidean= distance.euclidean(weightsDelta, oldweightsDelta)
        if distance_euclidean< errorAccuracy:
            print('Дошли до заданной ошибки точности на шаге: %d' % step)
            break

        weights=list(map(lambda w,wd: w+wd*k/l,weights,weightsDelta))

    print('Евклидово расстояние=%.6f' % distance_euclidean)
    return weights

-----------------------

In [75]:
from scipy.spatial  import distance
from sklearn.metrics import roc_auc_score
KK=GradientDescent(0,df)
KK

C=0
Дошли до заданной ошибки точности на шаге: 406
Евклидово расстояние=0.000010


[0.2881020967218675, 0.09171473755090646]

4. Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [77]:
w1, w2 = GradientDescent(0,df)
rw1, rw2 = GradientDescent(10,df)

C=0
Дошли до заданной ошибки точности на шаге: 406
Евклидово расстояние=0.000010
C=10
Дошли до заданной ошибки точности на шаге: 85
Евклидово расстояние=0.000010


5. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).

In [78]:
def sigm(X, w1, w2):
    return 1.0 / (1.0 + math.exp(-w1 * X['Sign1'] - w2 * X['Sign2']))
 
 
y_score = X.apply(lambda x: sigm(x, w1, w2), axis=1)
y_rscore = X.apply(lambda x: sigm(x, rw1, rw2), axis=1)
 
auc = roc_auc_score(y, y_score)
rauc = roc_auc_score(y, y_rscore)

print (auc, rauc)

(0.9267619047619047, 0.9370476190476189)


2 вариант функции

In [84]:
def GetAUC_ROC( C, data):
    weights= GradientDescent( C, data)
    y_true =  list(map( lambda y: 0  if y< 0  else 1 , data[ 0 ].values))#Класс объекта
    X_train =  data.loc[:, 1:]#Характеристики объектов

    y_scores= list(map( lambda x: 1  / ( 1  +  math.exp( - weights[ 0 ] * x[ 0 ] -  weights[ 1 ] * x[ 1 ])) , X_train.values))
    aUC_ROC= roc_auc_score( y_true, y_scores)
    print('AUC_ROC=%.3f' %  round( aUC_ROC, 3))
    print( ' - - - -')
    return  aUC_ROC

#выводим ответ
answ=[]
answ.append( GetAUC_ROC( 0, df))
answ.append( GetAUC_ROC( 2, df))
answ

C=0
Дошли до заданной ошибки точности на шаге: 406
Евклидово расстояние=0.000010
AUC_ROC=0.927
 - - - -
C=2
Дошли до заданной ошибки точности на шаге: 227
Евклидово расстояние=0.000010
AUC_ROC=0.934
 - - - -


[0.9267619047619047, 0.9337142857142856]

запись в файл ответа

In [79]:
file=open('3.logic_regression.txt','w')
file.write("{:0.3f},{:0.3f}".format(auc, rauc))
file.close()

  6. Попробуйте поменять длину шага. Будет ли сходиться алгоритм, если делать более длинные шаги? Как меняется число итераций при уменьшении длины шага?

7. Попробуйте менять начальное приближение. Влияет ли оно на что-нибудь?